## 1. LQR算法

### 优缺点分析
- 优点：
  - 如果系统是线性的，则LQR可以很好地控制系统到稳态（即states=0,当然states可以是误差）。
- 缺点：
  - 线性系统。
  - 难以处理路径限制，以及添加障碍物等。
  - 难以处理控制限制。

### *参考资料

> - [自动驾驶算法详解(3) : LQR算法进行轨迹跟踪](https://zhuanlan.zhihu.com/p/504299366)
> - [comp417/lqr_examples](https://github.com/florianshkurti/comp417/tree/master/lqr_examples)
> - [CSC2621 Imitation Learning for Robotics](http://www.cs.toronto.edu/~florian/courses/imitation_learning/lectures/Lecture2.pdf)
> - [LQR 的直观推导及简单应用](https://heyijia.blog.csdn.net/article/details/39270597?spm=1001.2101.3001.6650.8&utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7Edefault-8.pc_relevant_default&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7Edefault-8.pc_relevant_default&utm_relevant_index=12)
> - [Welcome to PythonRobotics’s documentation!](https://atsushisakai.github.io/PythonRobotics/index.html)
> - [PythonRobotics](https://github.com/AtsushiSakai/PythonRobotics)


### 1.1 算法推导<font color='red'>（有限时间、离散系统）</font>
考虑$t\in[t_0, t_1]$区间内的离散系统（线性系统）：
$$
x_{k+1} = Ax_k+Bu_k,\quad x_0 = x^{init}
$$
其中$x_k\in\mathbb{R}^n$, $u_k\in\mathbb{R}^m$.

二次cost function定义为：
$$
J = x^T_NQ_fx_N+\sum^{N-1}_{k=0}(x^T_kQx_k+u^T_kRu_k)
$$

记$V_{t}(z)$表示为从t时刻的状态z开始，通过最优控制序列，获得的到结束后的最小代价:
$$
V_t(z) = \min_{u_t,\dots,u_{N-1}}\sum^{N-1}_{\tau=t}(x^T_\tau Qx_\tau+u^T_\tau Ru_\tau) + x^T_NQ_fx_N
$$
则有：
$$
V_N(z) = z^TQ_fz
$$
同时，我们不难知道有如下的关系：
$$
V_t(z) = \min_{w}\{z^TQz+w^TRw+V_{t+1}(Az+Bw)\}
$$
其中$z^TQz+w^TRw$是基于当前时刻状态采用控制$w$的代价；$V_{t+1}(Az+Bw)$是下一时刻到最终时刻的代价。

我们<font color='red'>假设$V_t(z) = z^TP_tz$</font>，则上式可以改写为：
$$
V_t(z) = z^TQz + \min_{w}\{w^TRw+(Az+Bw)^TP_{t+1}(Az+Bw)\}
$$
为了求解$w$,只需要对上式右端对$w$求导并使其为0，从而得到：
$$
2w^TR+2(Az+Bw)^TP_{t+1}B = 0
$$
从而有：
<font color='blue'>
$$
w^* = -(R+B^TP_{t+1}B)^{-1}B^TP_{t+1}Az
$$
</font>
将其回代公式，有：
$$
\begin{align}
V_t(z) &= z^T(Q+A^TP_{t+1}A - A^TP_{t+1}B(R+B^TP_{t+1}B)^{-1}B^TP_{t+1}A)z \\
&= z^TP_t z
\end{align}
$$
从而得到：
<font color='red'>
$$
\begin{align}
P_t &= Q+A^TP_{t+1}A - A^TP_{t+1}B(R+B^TP_{t+1}B)^{-1}B^TP_{t+1}A \\
P_N &= Q_f
\end{align}
$$
</font>

对于控制$u^*$，从$w^*$表达式可知
<font color='red'>
$$
u_t = -K(t)x_t,\quad \text{其中} K(t) = (R+B^TP_{t+1}B)^{-1}B^TP_{t+1}A
$$
</font>


### 算法步骤
- <font color='blue'>Backward</font>:
  - 使$P_N = Q_f$
  - 对$t=N,\cdots,1$, 计算
$$
P_{t-1} = Q+A^TP_tA - A^TP_tB(R+B^TP_tB)^{-1}B^TP_tA 
$$

- <font color='blue'>Forward</font>:
  - 对$t=0,\cdots,N-1$，定义
$$
K_t = (R+B^TP_{t+1}B)^{-1}B^TP_{t+1}A
$$
  - 对$t=0,\cdots,N-1$，使
$$
u_t = -K_tx_t
$$



### 1.2 算法推导 <font color='red'>（无限时间、离散系统）</font>
- 在$t\in[0, +\infty]$区间内，离散格式动力系统为：
$$
x_{k+1} = Ax_k+Bu_k
$$
其中$x_k\in\mathbb{R}^n$, $u_k\in\mathbb{R}^m$.

- 二次cost function定义为：
$$
J = \sum^{\infty}_{k=0}(x^T_kQx_k+u^T_kRu_k)
$$

该种情形可以看成 <font color='blue'>（有限时间、离散系统）</font>的推广，当$t_1\longrightarrow \infty$时，$p(t)$趋于稳定，因此有$p(t)$趋于稳定不变，有如下Riccati方程
<font color='red'>
$$
P = Q+A^TPA - A^TPB(R+B^TPB)^{-1}B^TPA 
$$
    </font>
且从$w^*$表达式可知
<font color='red'>
$$
u_t = -Kx_t,\quad \text{其中} K = (R+B^TPB)^{-1}B^TPA
$$
</font>

### 1.3 算法推导 <font color='red'>（有限时间、连续系统）</font>
- 在$t\in[t_0, t_1]$区间内，动力系统为：
$$
\dot{x} = Ax+Bu
$$
其中$x\in\mathbb{R}^n$, $u\in\mathbb{R}^m$.

- 二次cost function定义为：
$$
J = x^T(t_1)Q_fx(t_1)+\int^{t_1}_{t_0}(x^TQx+u^TRu)dt
$$

则上函数的Hamiltonian为：
$$
H(x,\lambda,u) = x^TQx+u^TRu + \lambda^T\cdot(Ax+Bu)
$$
则通过PMP定理有：
$$
\begin{cases}
\dot{x} &= \partial^T_\lambda H(x,\lambda,u) \\
\dot{\lambda} &= -\partial^T_x H(x,\lambda,u
\end{cases}
\Longrightarrow 
\color{blue}{(*)}
\begin{cases}
\dot{x} & = Ax + Bu\\
\dot{\lambda} & = -(Q^T+Q)x - A^T \lambda
\end{cases}
$$
对Hamiltonian关于$u$求导并使其等于零有:
$$
\frac{\partial H}{\partial u} = (R^T+R)u + B^T\lambda = 0
\Longrightarrow
u^* = -(R^T+R)^{-1}B^T\lambda
$$
把$u^*$代入到$(*)$中，有
$$
\color{blue}{(1.1)}
\begin{cases}
\dot{x} & = Ax - B(R^T+R)^{-1}B^T\lambda\\
\dot{\lambda} & = -(Q^T+Q)x - A^T \lambda
\end{cases}
$$
边界条件为
$$
\color{blue}{(1.2)}
\begin{cases}
x(t_0) &= x_0 \\
\lambda(t_1) & = \frac{\partial M(x)}{\partial x} = (Q_f^T+Q_f)x(t_1)
\end{cases}
$$
则$x$、$\lambda$可以通过求解两点边值问题(TPBVP)<font color='blue'>(1.1)-(1.2)</font>得到。

我们<font color='red'>假设$\lambda$可以表示为$\lambda(t) = p(t)x(t)$</font>，将其代入到$(*)$中，可以得到
$$
\begin{align}
\dot{p}(t)x(t) + p(t)\dot{x}(t) & = -(Q^T+Q)x(t) - A^T p(t)x(t) \\
\Longrightarrow \dot{p}(t)x(t) + p(t)(Ax(t)+Bu^*(t)) &= -(Q^T+Q)x(t) - A^T p(t)x(t) 
\end{align}
$$

同时，由于$u^*(t) = -(R^T+R)^{-1}B^TP(t)x(t)$，进一步可以得到
$$
\dot{p}(t)x(t) + p(t)Ax(t)-P(t)B(R^T+R)^{-1}B^TP(t)x(t) = -(Q^T+Q)x(t) - A^T p(t)x(t) 
$$

约去$x(t)$，可得如下等式
<font color='red'>
$$
\color{blue}{(2.1)}\quad
(Q^T+Q) + A^T p(t) + p(t)A - P(t)B(R^T+R)^{-1}B^TP(t)  = -\dot{p}(t)
$$
</font>
由(1.2)可得
<font color='red'>
$$
\color{blue}{(2.2)}\quad
p(t_1) = (Q^T_f + Q_f)
$$
</font>
其中，<font color='blue'>(2.1)-(2.2)</font>即为Riccati方程。控制$u^*$的表达式为
<font color='red'>
$$
\begin{cases}
u^*(t) = -K(t)x(t) \\
其中\ K(t) = (R^T+R)^{-1}B^TP(t)
\end{cases}
$$
</font>

### 1.4 算法推导 <font color='red'>（无限时间、连续系统）</font>
- 在$t\in[0, +\infty]$区间内，动力系统为：
$$
\dot{x} = Ax+Bu
$$
其中$x\in\mathbb{R}^n$, $u\in\mathbb{R}^m$.

- 二次cost function定义为：
$$
J = \int^{\infty}_{0}(x^TQx+u^TRu)dt
$$

该种情形可以看成 <font color='blue'>（有限时间、连续系统）</font>的推广，当$t_1\longrightarrow \infty$时，$p(t)$趋于稳定，因此有$\dot{p}(t)=0$。从而得到
<font color='red'>
$$
(Q^T+Q) + A^T p + p A - P B(R^T+R)^{-1}B^TP = 0
$$
</font>
相应地，有
<font color='red'>
$$
\begin{cases}
u^* = -K x(t) \\
其中\ K = (R^T+R)^{-1}B^TP
\end{cases}
$$
</font>

## 2. 各种情形下的LQR算法汇总

### 2.1 有限时域、离散系统

- 在$t\in[t_0, t_1]$区间内，离散格式动力系统为：
$$
x_{k+1} = Ax_k+Bu_k
$$
其中$x_k\in\mathbb{R}^n$, $u_k\in\mathbb{R}^m$.

- 二次cost function定义为：
$$
J = x^T_NQ_fx_N+\sum^{N-1}_{k=0}(x^T_kQx_k+u^T_kRu_k+2x^T_kNu_k)
$$

- Feedback control law为：
$$
u_k = -K_k x_k,\quad \text{其中} K_k= (R+B^TP_{k+1}B)^{-1}(B^TP_{k+1}A+N^T)
$$

- Riccati方程为：
$$
\begin{align}
P_{k-1} &= Q+A^TP_kA - (A^TP_kB+N)(R+B^TP_kB)^{-1}(B^TP_kA+N^T)  \\
P_N &= Q_f
\end{align}
$$
> - <font color='red'>  $K_k$是依赖于时间的；</font>
> - <font color='red'>  $P_k$有边界条件。 </font>

### 2.2 有限时域、连续系统

- 在$t\in[t_0, t_1]$区间内，动力系统为：
$$
\dot{x} = Ax+Bu
$$
其中$x\in\mathbb{R}^n$, $u\in\mathbb{R}^m$.

- 二次cost function定义为：
$$
J = x^T(t_1)Q_fx(t_1)+\int^{t_1}_{t_0}(x^TQx+u^TRu+2x^TNu)dt
$$

- Feedback control law为：
$$
u = -K(t)x,\quad \text{其中} K(t)= R^{-1}(B^TP(t)+N^T)
$$

- Riccati方程为：
$$
\begin{align}
-\dot{P}(t) &= Q + A^TP(t) + P(t)A - (P(t)B+N)R^{-1}(B^TP(t)+N^T) \\
P(t_1)& = Q_f
\end{align}
$$

> - <font color='red'>  K(t)是依赖于时间的；</font>
> - <font color='red'>  P(t)有边界条件。 </font>

### 2.3 无限时域、离散系统

- 在$t\in[0, +\infty]$区间内，离散格式动力系统为：
$$
x_{k+1} = Ax_k+Bu_k
$$
其中$x_k\in\mathbb{R}^n$, $u_k\in\mathbb{R}^m$.

- 二次cost function定义为：
$$
J = \sum^{\infty}_{k=0}(x^T_kQx_k+u^T_kRu_k+2x^T_kNu_k)
$$

- Feedback control law为：
$$
u_k = -Kx_k,\quad \text{其中} K= (R+B^TPB)^{-1}(B^TPA+N^T)
$$

- Riccati方程为：
$$
\begin{align}
P = Q+A^TPA - (A^TPB+N)(R+B^TPB)^{-1}(B^TPA+N^T) 
\end{align}
$$
> - <font color='red'>  K不依赖于时间；</font>
> - <font color='red'>  P稳定。 </font>


### 2.4 无限时域、连续系统
- 在$t\in[0, +\infty]$区间内，动力系统为：
$$
\dot{x} = Ax+Bu
$$
其中$x\in\mathbb{R}^n$, $u\in\mathbb{R}^m$.

- 二次cost function定义为：
$$
J = \int^{\infty}_{0}(x^TQx+u^TRu+2x^TNu)dt
$$

- Feedback control law为：
$$
u = -Kx,\quad \text{其中} K= R^{-1}(B^TP+N^T)
$$

- Riccati方程为：
$$
\begin{align}
Q + A^TP + PA - (PB+N)R^{-1}(B^TP+N^T) = 0
\end{align}
$$


> - <font color='red'>  K不依赖于时间；</font>
> - <font color='red'>  P稳定。 </font>


## 3 数值算例

### 3.1 omnidirectional vehicle

- **连续动力学系统：**
$$
m\ddot{p} = u - \alpha \dot{p} 
$$
其中$p\in\mathbb{R}^2$表示position；$u\in\mathbb{R}^2$表示控制；$\alpha$表示摩擦系数。

记$v=\dot{p}$,则有
$$
\begin{cases}
\dot{p} &= v \\
\dot{v} &= \frac{1}{m}u-\frac{\alpha}{m}v 
\end{cases}
$$

- **离散动力学系统：**
我们记$X_t=(P_t, v_t)$，则有
$$
X_{t+1} = AX_t + Bu_t
$$
其中
$$
A = 
\left[
\begin{array}{cccc}
1 & 0 & dt & 0 \\
0 & 1 & 0  & dt \\
0 & 0 & 1-dt*\frac{\alpha}{m} & 0 \\
0 & 0 & 0 & 1-dt*\frac{\alpha}{m}
\end{array}
\right]
,\quad
B = 
\left[
\begin{array}{cc}
0 & 0 \\
0 & 0 \\
\frac{dt}{m} & 0 \\
0 & \frac{dt}{m} 
\end{array}
\right]
$$

### 3.2 情形（1）: <font color='blue'>线性系统 + 无xd + 无ud</font>
- $\textbf{一般描述}：$对于该情形，它的dynamics方程为
$$
\textbf{x}_{t+1} = A\textbf{x}_t + B \textbf{u}_t
$$

Cost function 为
$$
\begin{align}
c(x,u) & = x^TQx + u^TRu \\
       & = x^Tx + \rho u^Tu \\
       & = \|x\|^2+\rho \|u\|^2
\end{align}
$$

- $\textbf{处理方式：}$直接使用LQR算法，无需特殊处理。

### 3.3 情形（2）: <font color='blue'>线性系统 + <font color='red'>有xd 或 ud</font> </font>
- $\textbf{一般描述}：$ 对于该情形，它的dynamics方程为 
$$
\textbf{x}_{t+1} = A\textbf{x}_t + B \textbf{u}_t
$$
Cost function 为
$$
c(x,u) = (x - x^d)^TQ(x - x^d) + u^TRu
$$
其中$x^d(t)$为给定的参考路径（即追踪路径）。

- $\textbf{处理方式}：$ 对于该情形，则我们引入一个新变量$\bar{x}(t)$，使
$$
\begin{align}
\bar{x} & = x_{t+1} - x^*_{t+1} \\
        & = Ax_{t} + Bu_t - x^*_{t+1} \\
        & = A\bar{x}_t + Bu_t - x^*_{t+1} + Ax^*_t
\end{align}
$$
为了转化为$x_{t+1} = Ax_t+Bu_t$的形式，我们记变量$\textbf{z}_t$
$$
\begin{equation}
z_{t+1} = 
\left [
\begin{array}{cc}
\bar{x}_{t+1} \\
1
\end{array}
\right] =
\left [
\begin{array}{cc}
A &  {\color{red}h}\\
0 & 1
\end{array}
\right]
\left [
\begin{array}{cc}
\bar{x}_t\\
1
\end{array}
\right]
+
\left [
\begin{array}{cc}
B \\
0
\end{array}
\right] u_t
=
\bar{A}z_t + \bar{B}u_t
\end{equation}
$$
其中${\color{red}h = -x^*_{t+1}+Ax^*_{t}}$. Cost function 为
$$
c(z_t, u_t) = z^T_tQz_t + u^T_tRu_t
$$

### 3.3 情形（3）: <font color='blue'><font color='red'>非线性系统</font>  + 无xd+ 无ud</font>
- $\textbf{一般描述}$：对于该情形，它的dynamics方程为
$$
\textbf{x}_{t+1} = f(x_t, u_t)
$$
Cost function 为
$$
c(x,u) = x^TQx + u^TRu
$$


- $\textbf{处理方式}$： 对于该情形，我们需要对非线性系统进行线性化，其中<font color='blue'>最关键的是在何处（即$x^d$和$u^d$）进行线性化？</font> 由于我们的cost希望x越小越好，因此我们使$x^d = \textbf{0}$，而$u^d$通过求解如下方程获得 
<font color='blue'>
$$
\textbf{0}_{t+1} = f(\textbf{0}_{t}, u^d)
$$
</font>
然后我们对dynamics方程在$(\textbf{0}, u^d)$处进行展开：
$$
x_{t+1}\approx f(\textbf{0}, u^d) + \frac{\partial f}{\partial x}(\textbf{0}, u^d)(x_t - \textbf{0}) + \frac{\partial f}{\partial u}(\textbf{0}, u^d)(u_t - u^d)
$$
我们记
$$
A = \frac{\partial f}{\partial x}（\textbf{0}, u^d),\quad 
B = \frac{\partial f}{\partial u} (\textbf{0}, u^d)
$$
因此，我们得到了新的动力学方程
$$
x_{t+1} \approx A x_t + B(u_t - u^d)
$$
对应的cost依然为
$$
c(x_t, u_t) = x^T_tQx_t + u^T_tRu_t
$$

### 3.3 情形（4）: <font color='red'>非线性系统 + 有xd+ 有ud</font>
- $\textbf{一般描述}：$ 对于该情形，它的dynamics方程为
$$
\textbf{x}_{t+1} = f(x_t, u_t)
$$
Cost function 为
$$
c(x,u) = (x-x^d)^TQ(x-x^d) + (u-u^d)^TR(u-u^d)
$$
其中$x^d(t)$和$u^d(t)$分别为给定的参考路径和参考控制。

- $\textbf{处理方式}：$ 对于该情形，我们对非线性系统进行线性化，即对dynamics方程在$(x^d, u^d)$处进行展开：
$$
x_{t+1}\approx f(x^d, u^d) + \frac{\partial f}{\partial x}(x^d, u^d)(x_t - x^d) + \frac{\partial f}{\partial u}(x^d, u^d)(u_t - u^d)
$$
我们记
$$
\bar{x}_t = x_t - x^d_t,\quad
\bar{u}_t = u_t - u^d_t \\
$$
$$
A_t = \frac{\partial f}{\partial x}（x^d, u^d),\quad 
B_t = \frac{\partial f}{\partial u} (x^d, u^d)
$$
因此，我们得到了新的动力学方程
$$
\bar{x}_{t+1} \approx \bar{A}_t \bar{x}_t + \bar{B}_t \bar{u}_t
$$
对应的cost则为
$$
c(\bar{x}_t, \bar{u}_t) = \bar{x}^T_tQ\bar{x}_t + \bar{u}^T_tR\bar{u}_t
$$

## 4 数值算例2: 自动驾驶小车
车辆运动模型如下：
$$
\begin{cases}
\dot{x} &=  v\cos(\psi) \\
\dot{y} &= v\sin(\psi) \\
\dot{\psi} &= \frac{v}{L}\tan(\delta) \\
\dot{v}&= a
\end{cases}
$$

#### 模型线性化：
$$
\begin{equation}
\color{blue}{(*): }
\dot{X} =
\left[ \begin{array}{cc}
        \dot{x} \\
        \dot{y} \\
        \dot{\psi} \\
        \dot{v}
        \end{array}
\right] = 
\left[ \begin{array}{cc}
        v\cos(\psi) \\
        v\sin(\psi) \\
        \frac{v\tan{\delta}}{L} \\
        a
        \end{array}
\right] 
= f(X,u)
\end{equation}
$$
其中$X=[x,y,\psi,v]^T,\ u=[a, \delta]^T$.

将$(*)$在$X_r=[x_r,y_r,\psi_r,v_r]^T, u_r=[a_r, \delta_r]^T$处展开并忽略二阶和高阶项，可得：
$$
\begin{align}
\dot{X} = f(X,u) &\approx f(X_r, u_r) + \frac{\partial f}{\partial X_r}(X-X_r) 
+ \frac{\partial f}{\partial u_r}(u-u_r) \\
& = \dot{X}_r + A_t(X-X_r) + B_t(u-u_r)
\end{align}
$$
其中
$$
A_t =
\left[
\begin{array}{cccc}
0 & 0 & -v_r\sin{\psi_r}& \cos{\psi_r}\\
0 & 0 & v_r\cos{\psi_r}& \sin{\psi_r}\\
0 & 0 & 0 & \frac{\tan{\delta_r}}{L}\\
0 & 0 & 0 & 0
\end{array}
\right],
\quad
B_t =
\left[
\begin{array}{cc}
0 & 0  \\
0 & 0  \\
0 & \frac{v_r}{L\cos^2{\delta_r}} \\
1 & 0
\end{array}
\right]
$$

记<font color='red'>$\Delta X=X-X_r,\ \Delta u =u-u_r$</font>, 则有
$$
\color{red}{(*):\quad \Delta \dot{X} = A_t\Delta X + B_t \Delta u}
$$

#### 线性模型离散化：
我们对$(*)$进行离散化，假设时间步长为$dt$，则有
$$
\begin{align}
\Delta X_{t+1} &= (I+dtA_t)\Delta X_t + dtB_t\Delta u_t \\
 &= \tilde{A}_t \Delta X_t + \tilde{B}_t \Delta u_t.
\end{align}
$$
其中
$$
\tilde{A}_t = I+dtA_t 
=\left[
\begin{array}{cccc}
1 & 0 & -v_r\sin{\psi_r}dt & \cos{\psi_r}dt\\
0 & 1 & v_r\cos{\psi_r}dt& \sin{\psi_r}dt\\
0 & 0 & 1 & \frac{\tan{\delta_r}}{L}dt\\
0 & 0 & 0 & 1
\end{array}
\right]
,
\quad \tilde{B}_t = dtB_t 
=\left[
\begin{array}{cc}
0 & 0  \\
0 & 0  \\
0 & \frac{v_r}{L\cos^2{\delta_r}}dt \\
dt & 0
\end{array}
\right]
.
$$


### 4.1：路径追踪问题

#### <font color='red'>（1）如何选取$u_r$ ?(注意一下这里)</font>
在$X_r=(x_r, y_r, \psi_r, v_r)$的情况下，我们选择$u_r=(a_r, \delta_r)$使得如下方程成立：

$$
\begin{cases}
\dot{x}_r &=  v_r\cos(\psi_r) \\
\dot{y}_r &= v_r\sin(\psi_r) \\
\dot{\psi}_r &= \frac{v_r}{L}\tan(\delta_r) \\
\dot{v}_r & = a_r
\end{cases}
$$
则可以得到
<font color='red'>
$$
\begin{cases}
a_r &= \dot{v}_r\\
\delta_r &=\arctan{\frac{L\cdot\dot{\psi}_r}{v_r}}
\end{cases}
$$
</font>

#### （2）问题分析：

<font color='red'>我们设置$u_r = [a_r, \delta_r] = [0., 0.]$</font>，则有
$$
\begin{align}
\Delta X_{t+1} = \tilde{A}_t \Delta X_t + \tilde{B}_t u_t.
\end{align}
$$
其中
$$
\tilde{A}_t = I+dtA_t 
=\left[
\begin{array}{cccc}
1 & 0 & -v_r\sin{\psi_r}dt & \cos{\psi_r}dt\\
0 & 1 & v_r\cos{\psi_r}dt& \sin{\psi_r}dt\\
0 & 0 & 1 & 0\\
0 & 0 & 0 & 1
\end{array}
\right]
,
\quad \tilde{B}_t = dtB_t 
=\left[
\begin{array}{cc}
0 & 0  \\
0 & 0  \\
0 & \frac{v_r}{L}dt \\
dt & 0
\end{array}
\right]
.
$$

但其中的<font color='red'>$A_t, B_t$依赖于$X_r$中的$v_r$和$\psi_r$</font>.

#### （3）**代码实现：**

> - <font color='blue'>模型为： uav_car.py </font>
> - <font color='blue'>有限时间、离散情形：uav_car_LQR_a.py (速度快，但转弯时不太好)</font>
> - <font color='blue'>无限时间、离散情形：uav_car_LQR_b.py (速度慢，效果好)</font>
> - <font color='red'>当计算$A_t, B_t$却无法提供$\psi_r$时，利用$\psi_t$代替可获得相对较好的效果。</font>


## 5 数值算例3: 自动驾驶小车
考虑简化车辆运动模型如下（不考虑控制加速度a）：
$$
\begin{cases}
\dot{x} &=  v\cos(\psi) \\
\dot{y} &= v\sin(\psi) \\
\dot{\psi} &= \frac{v}{L}\tan(\delta) 
\end{cases}
$$

#### 模型线性化：
$$
\begin{equation}
\color{blue}{(*): }
\dot{X} =
\left[ \begin{array}{cc}
        \dot{x} \\
        \dot{y} \\
        \dot{\psi} 
        \end{array}
\right] = 
\left[ \begin{array}{cc}
        v\cos(\psi) \\
        v\sin(\psi) \\
        \frac{v\tan{\delta}}{L} 
        \end{array}
\right] 
= f(X,u)
\end{equation}
$$
其中$X=[x,y,\psi]^T,\ u=[v, \delta]^T$.

将$(*)$在$X_r=[x_r,y_r,\psi_r]^T, u_r=[v_r, \delta_r]^T$处展开并忽略二阶和高阶项，可得：
$$
\begin{align}
\dot{X} = f(X,u) &\approx f(X_r, u_r) + \frac{\partial f}{\partial X_r}(X-X_r) 
+ \frac{\partial f}{\partial u_r}(u-u_r) \\
& = \dot{X}_r + A_t(X-X_r) + B_t(u-u_r)
\end{align}
$$
其中
$$
A_t =
\left[
\begin{array}{cccc}
0 & 0 & -v_r\sin{\psi_r}\\
0 & 0 & v_r\cos{\psi_r}\\
0 & 0 & 0 
\end{array}
\right],
\quad
B_t =
\left[
\begin{array}{cc}
\cos(\psi_r) & 0  \\
\sin(\psi_r) & 0  \\
\frac{\tan(\delta_r)}{L} & \frac{v_r}{L\cos^2{\delta_r}} \\
\end{array}
\right]
$$

记<font color='red'>$\Delta X=X-X_r,\ \Delta u =u-u_r$</font>, 则有
$$
\color{red}{(*):\quad \Delta \dot{X} = A_t\Delta X + B_t \Delta u}
$$

#### 线性模型离散化：
我们对$(*)$进行离散化，假设时间步长为$dt$，则有
$$
\begin{align}
\Delta X_{t+1} &= (I+dtA_t)\Delta X_t + dtB_t\Delta u_t \\
 &= \tilde{A}_t \Delta X_t + \tilde{B}_t \Delta u_t.
\end{align}
$$
其中
$$
\tilde{A}_t = I+dtA_t 
=\left[
\begin{array}{cccc}
1 & 0 & -v_r\sin{\psi_r}dt \\
0 & 1 & v_r\cos{\psi_r}dt \\
0 & 0 & 1 & \\
\end{array}
\right]
,
\quad \tilde{B}_t = dtB_t 
=\left[
\begin{array}{cc}
\cos{\psi_r}dt & 0  \\
\sin{\psi_r}dt& 0  \\
\frac{\tan{\delta_r}}{L}dt & \frac{v_r}{L\cos^2{\delta_r}}dt 
\end{array}
\right]
.
$$


### 5.1：路径追踪问题

#### <font color='red'>（1）如何选取$u_r$ ?（注意一下这里）</font>
在$X_r=(x_r, y_r, \psi_r)$的情况下，我们选择$u_r=(v_r, \delta_r)$使得如下方程成立：

$$
\begin{cases}
\dot{x}_r &=  v_r\cos(\psi_r) \\
\dot{y}_r &= v_r\sin(\psi_r) \\
\dot{\psi}_r &= \frac{v_r}{L}\tan(\delta_r) 
\end{cases}
$$
则可以得到
<font color='red'>
$$
\begin{cases}
v_r &= \sqrt{\dot{x}_r^2 + \dot{y}_r^2}\\
\delta_r &=\arctan{\frac{L\cdot\dot{\psi}_r}{v_r}}
\end{cases}
$$
</font>

#### （2）问题分析：
<font color='red'>我们设置$u_r = [v_r, \delta_r] = [2., 0.]$</font>，则有
$$
\begin{align}
\Delta X_{t+1} = \tilde{A}_t \Delta X_t + \tilde{B}_t \Delta u_t.
\end{align}
$$
其中
$$
\tilde{A}_t = I+dtA_t 
=\left[
\begin{array}{cccc}
1 & 0 & -v_r\sin{\psi_r}dt \\
0 & 1 & v_r\cos{\psi_r}dt \\
0 & 0 & 1 & \\
\end{array}
\right]
,
\quad \tilde{B}_t = dtB_t 
=\left[
\begin{array}{cc}
\cos{\psi_r}dt & 0  \\
\sin{\psi_r}dt& 0  \\
0 & \frac{v_r}{L}dt 
\end{array}
\right]
.
$$

但其中的<font color='red'>$A_t, B_t$依赖于$X_r$中的$\psi_r$</font>.

#### （3）**代码实现**

> - <font color='blue'>模型为： uav_car_simple.py </font>
> - <font color='blue'>有限时间、离散情形：uav_car_simple_LQR_a.py (速度快，但转弯时不太好)</font>
> - <font color='blue'>无限时间、离散情形：uav_car_simple_LQR_b.py (速度相对较慢，但效果非常好)</font>
> - <font color='red'>当计算$A_t, B_t$却无法提供$\psi_r$时，利用$\psi_t$代替可获得相对较好的效果。</font>（在代码uav_car_simple_LQR_b.py中取消$\psi_r$替换代码的注释即可）
